In [48]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pathresults = "../../results/"
#pathresults = "/DUMUXDune27/DUMUX/dumux-rosi/python/paperSc/results/"
data_file_delimiter = ","
import re
countryRoad = r'newMucil/none_1476_{}_10to14_20mn_0s_10/'

In [44]:
def getValT(mypath, filename, header="infer",names = None):
    fullpath = pathresults +mypath+filename
    if names is None:
        cst = pd.read_csv(fullpath,delimiter=data_file_delimiter, header = header).dropna(how= "all", axis=1)
    else:
        cst = pd.read_csv(fullpath,delimiter=data_file_delimiter, header = header, names = names).dropna(how= "all", axis=1)
    return cst

exceptPset = []
exceptPsetTime = []
for i in range(99):
    # path2file = pathresults + 'param99none'+str(i)+'0dx_2dumux_10c_10.0to11.0_20mn_0s_192_100/'
    path2file = countryRoad.format(i)#'newMucil/none_1476_'+str(i)+'_10to14_20mn_0s_10/'
    #'none_55_'+str(i)+'_10to11_20mn_0s_5/'# 'fixeUpdatenone'+str(i)+'0dx_2dumux_10c_10.0to25.0_20mn_0s_192_100/'
    tt = getValT(path2file, "time.txt",names= ['time','Qlight'])
    time = tt.iloc[:,0]
    if max(time) < 14: 
        exceptPset.append(i)
        exceptPsetTime.append(max(time))
        print(i, (int((max(time)-10.)*24*10)/10), end ="h, ")
print('\n',len(exceptPset))#, (int((min(exceptPsetTime)-10.)*24*10)/10))


 0


In [54]:
def getVal(mypath, filename, header="infer",names = None):
    fullpath = pathresults +mypath+filename
    if names is None:
        cst = pd.read_csv(fullpath,delimiter=data_file_delimiter, header = header).dropna(how= "all", axis=1)
    else:
        cst = pd.read_csv(fullpath,delimiter=data_file_delimiter, header = header, names = names).dropna(how= "all", axis=1)
    return cst
def get_last_line_as_numpy_array(file_path, dtype = float):
    with open(pathresults +file_path, 'r') as file:
        # Read all lines
        lines = file.readlines()

        # Check if there are any lines in the file
        if not lines:
            raise ValueError("The file is empty")

        # Get the last line
        last_line = lines[-1].strip()

        # Split the last line by commas and convert to NumPy array
        array_from_last_line = np.array(last_line.split(','), dtype=dtype)

        return array_from_last_line

def list_files_with_prefix(folder_path, prefix):
    file_names = []
    for file_name in os.listdir(pathresults+folder_path):
        if file_name.startswith(prefix):
            file_names.append(file_name)
    return file_names
def getCylIdx(folder_path):
    namelist = list_files_with_prefix(folder_path+"cyl_val/", "Cyl_watercontent_")
    idxlist = np.full(len(namelist),-1,dtype = int)
    for nn, nl in enumerate(namelist):
        match = re.search(r'\d+', nl)
        number_str = match.group()
        number_int = int(number_str)
        idxlist[nn] = number_int
    return idxlist
# cylinder max konz (per cell)
def getData_(cid,gId,path2file):
    fileName = "cyl_val/Cyl_cellVol_"+str(gId)+".txt"
    cVol = get_last_line_as_numpy_array(path2file+fileName, dtype = float)
    if cid == volIdx:
        return sum(cVol)
    elif cid <= 9:
        if cid <= 2:# == 0:
            fileName = "cyl_val/Cyl_watercontent_"+str(gId)+".txt"
            theta = get_last_line_as_numpy_array(path2file+fileName, dtype = float)
            cVol *= theta#cm3 scv to cm3 water
            if cid == 0:
                return sum(cVol)
        fileName = "cyl_val/Cyl_content"+str(cid)+"_"+str(gId)+".txt"
        Q1 = get_last_line_as_numpy_array(path2file+fileName, dtype = float)
        konz = sum(Q1)/sum(cVol) #mol/cm3 or cm3/cm3
        return konz
    raise Exception

numPset = 99
nKonz = 9
dictXYZ = {9:'X',10:'Y',11:'Z'}
volIdx=12
lenIdx=13
stIdx=14
orgIdId=15
relLenId=16

nToGet = relLenId+1 #elements + 3d coordinates of y , node+vol+len+st 

exceptPset =[]
allPSets = [ i for i in range(numPset) if i not in exceptPset]

def getData():
    
    GiniAll = [[[] for i in range(numPset)] for ii in range(nToGet)]
    for pSet in allPSets:
        print(pSet,end =", ")

        #path2file = 'none_55_'+str(pSet)+'_10to11_20mn_0s_5/'
        path2file =  countryRoad.format(pSet)#'newMucil/none_1476_'+str(i)+'_10to14_20mn_0s_10/'
        
        rr = getCylIdx(path2file)
        rr.sort()
        print('(',len(rr),')',end =", ")

        for cid in range(nKonz):
            Ginits = np.array([getData_( cid, gId,path2file) for gId in rr])          
            GiniAll[cid][pSet] = Ginits
        for cid in range(nKonz,volIdx):
            nodes_ = get_last_line_as_numpy_array(path2file+"nodes_"+dictXYZ[cid]+".txt", dtype =float)
            GiniAll[cid][pSet] = nodes_[rr+1]#seg idx to y-idx
        #cid = nToGet -1
        Ginits = np.array([getData_( volIdx, gId,path2file) for gId in rr])
        GiniAll[volIdx][pSet] = Ginits
        lens = get_last_line_as_numpy_array(path2file+"length_Segs"+".txt", dtype =float)
        GiniAll[lenIdx][pSet] = lens[rr]
        sts = get_last_line_as_numpy_array(path2file+"subTypes"+".txt", dtype =float)
        assert len(sts) == len(lens)
        GiniAll[stIdx][pSet] = sts[rr]
        
        orgidPerNode = get_last_line_as_numpy_array(path2file+"orgidPerNode.txt", dtype =int)
        globalNodeId = get_last_line_as_numpy_array(path2file+"globalNodeId.txt", dtype =int)
        parentNidx = get_last_line_as_numpy_array(path2file+"parentNidx.txt", dtype =int)
        ot_orgs = get_last_line_as_numpy_array(path2file+"ot_orgs.txt", dtype =int)
        id_orgs = get_last_line_as_numpy_array(path2file+"id_orgs.txt", dtype =int)
        id_roots = id_orgs[np.where(ot_orgs==2)]
        
        org2basenode = dict(zip(id_orgs, parentNidx))
        #org2ot = dict(zip(id_orgs, ot_orgs))
        isRoot = np.array([oo in id_roots for oo in orgidPerNode])
        
        orgidPerRoot = orgidPerNode[np.where(isRoot)]
        globalNodeIdroot =  globalNodeId[np.where(isRoot)]        
        segIdxRoot = np.array([rr for _, rr in enumerate(globalNodeIdroot) if rr != org2basenode[orgidPerRoot[_]]]) -1
        orgIdxRoot = np.array([orgidPerRoot[_] for _, rr in enumerate(globalNodeIdroot) if rr != org2basenode[orgidPerRoot[_]]])
        #segIdx2ot = dict(zip(segIdxRoot,orgIdxRoot))
        
        relLens = np.zeros(len(segIdxRoot))
        #lens = get_last_line_as_numpy_array(path2file+"length_Segs"+".txt", dtype =float)
        for myorg in orgIdxRoot:
            thensegs = np.where(orgIdxRoot == myorg)#segIdxRoot[] 
            lenSegs = GiniAll[lenIdx][pSet]
            relLens[thensegs] = np.cumsum(lenSegs[thensegs])
            
            
        GiniAll[orgIdId][pSet] = orgIdxRoot
        GiniAll[relLenId][pSet] = relLens

    return GiniAll #cid pSet rr
    

In [55]:
GiniAll = getData()

0, ( 323 ), 1, ( 328 ), 2, ( 323 ), 3, ( 328 ), 4, ( 353 ), 5, ( 323 ), 6, ( 322 ), 7, ( 355 ), 8, ( 323 ), 9, ( 329 ), 10, ( 323 ), 11, ( 322 ), 12, ( 323 ), 13, ( 323 ), 14, ( 323 ), 15, ( 339 ), 16, ( 327 ), 17, ( 323 ), 18, ( 329 ), 19, ( 338 ), 20, ( 322 ), 21, ( 324 ), 22, ( 327 ), 23, ( 329 ), 24, ( 342 ), 25, ( 319 ), 26, ( 328 ), 27, ( 322 ), 28, ( 327 ), 29, ( 328 ), 30, ( 327 ), 31, ( 323 ), 32, ( 323 ), 33, ( 327 ), 34, ( 319 ), 35, ( 331 ), 36, ( 328 ), 37, ( 336 ), 38, ( 323 ), 39, ( 323 ), 40, ( 327 ), 41, ( 338 ), 42, ( 319 ), 43, ( 342 ), 44, ( 323 ), 45, ( 327 ), 46, ( 339 ), 47, ( 337 ), 48, ( 323 ), 49, ( 342 ), 50, ( 342 ), 51, ( 358 ), 52, ( 328 ), 53, ( 327 ), 54, ( 323 ), 55, ( 341 ), 56, ( 323 ), 57, ( 328 ), 58, ( 344 ), 59, ( 328 ), 60, ( 345 ), 61, ( 328 ), 62, ( 354 ), 63, ( 328 ), 64, ( 341 ), 65, ( 323 ), 66, ( 329 ), 67, ( 328 ), 68, ( 323 ), 69, ( 328 ), 70, ( 358 ), 71, ( 324 ), 72, ( 346 ), 73, ( 356 ), 74, ( 323 ), 75, ( 323 ), 76, ( 327 ), 77, ( 323

In [131]:
#assert np.array(GiniAll).shape == (nToGet, numPset, 232) # some have a couple of segments more

/tmp/ipykernel_1126/1273250247.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  assert np.array(GiniAll).shape == (nToGet, numPset, 232) #all same shape


AssertionError: 

In [19]:
np.array(GiniAll).shape

/tmp/ipykernel_324427/2873905747.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(GiniAll).shape


(17, 99)

In [56]:
# get param set
paramsets =  pd.read_csv('../../output_random_rows.csv')
paramsets = paramsets.drop(paramsets.columns.tolist()[-1],axis=1)
paramsets['k_C,S'] /=paramsets['theta']
paramsets['k_O,S'] /=paramsets['theta']
paramsets.drop(labels='theta', inplace=True, axis=1)
outputvalsname =  ['wat','cs','cl','coa','cod','cca',
                   'ccd','css2','co2','yX','yY','yZ','vol','lenSeg',
                   'st','orgId','relLen']
assert len(outputvalsname) == nToGet 
column_names = outputvalsname + paramsets.columns.tolist()+['pSet']
df = pd.DataFrame(columns=column_names)

for pset in allPSets:#range(numPset):
    N = GiniAll[0][pset].shape[0]
    #assert N == 232
    rows_to_duplicate_df = pd.DataFrame(paramsets.iloc[pset]).T
    duplicated_rows_df = rows_to_duplicate_df.loc[np.repeat(rows_to_duplicate_df.index.values, N)]
    # Duplicate the specific rows N times
    #print('init dupl row shape',duplicated_rows_df.shape)
    for jj, outVn in enumerate(outputvalsname):
        duplicated_rows_df[outVn] = GiniAll[jj][pset]
        #print('adding',jj,outVn,duplicated_rows_df.shape,len(GiniAll[jj][pset]))
    duplicated_rows_df['pSet'] = pset
    
    path2file =countryRoad.format(pset)#'newMucil/none_1476_'+str(pset)+'_10to11_20mn_0s_10/'
    simTime = get_last_line_as_numpy_array(path2file+"totalComputetime.txt", dtype =float)[0]
    
    duplicated_rows_df['simTime'] = simTime
    # Concatenate the original DataFrame with the duplicated specific rows
    df = pd.concat([df, duplicated_rows_df], ignore_index=True)
df['co'] = df['coa']+df['cod']
df['cc'] = df['cca']+df['ccd']

In [23]:
df['simTime'] 

0        3662.203318
1        3662.203318
2        3662.203318
3        3662.203318
4        3662.203318
            ...     
19468    5226.824289
19469    5226.824289
19470    5226.824289
19471    5226.824289
19472    5226.824289
Name: simTime, Length: 19473, dtype: float64

In [58]:
df.to_pickle('./cyl10to11_4d.pkl')

In [53]:
df['cs']

0        9.823151e-07
1        2.261230e-06
2        1.852822e-06
3        1.837276e-06
4        1.761422e-06
             ...     
33160    2.747954e-07
33161    1.104116e-06
33162    3.577060e-07
33163    3.733705e-07
33164    2.118259e-07
Name: cs, Length: 33165, dtype: float64

In [57]:
df['cs']

0        1.769464e-06
1        3.656621e-06
2        2.904982e-06
3        2.886543e-06
4        2.977854e-06
             ...     
32882    2.747954e-07
32883    1.104116e-06
32884    3.577060e-07
32885    3.733705e-07
32886    2.118259e-07
Name: cs, Length: 32887, dtype: float64